# Markov KDM Applications

In this notebook, we leverage KDM (Kernel Density Matrices) to perform Bayesian network inference specifically for categorization problems. KDM models provide a flexible approach to estimating probability distributions, which is essential for the effective application of Bayesian networks. By integrating KDM, we can capture complex dependencies and interactions within the data, enabling more accurate and robust inferences for categorization tasks. This approach allows for seamless handling of continuous and discrete variables, making it well-suited for a wide range of real-world applications.

## Clone Base Repository

In [3]:
!git clone https://github.com/ccgomezn/kdm.git

fatal: destination path 'kdm' already exists and is not an empty directory.


In [4]:
mv kdm kdm_orig

mv: rename kdm to kdm_orig/kdm: Directory not empty


In [5]:
cp -r kdm_orig//* .

## Import Libraries

In [3]:
from kdm.models import KDMSequentialJointClassModel, KDMGraphModel
from kdm.layers import CosineKernelLayer
import kdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
from keras import metrics
from keras import losses
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Dense
import keras
from pandas import read_csv, DataFrame
import os
import keras
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ReduceLROnPlateau



## Energy Consumption

In this section, we utilize a dataset detailing the consumption of energy in Italy to predict the emissions of various gases. By analyzing the relationship between energy usage and gas emissions, we aim to build predictive models that can help understand and mitigate the environmental impact of energy consumption.

The dataset includes variables such as energy consumption by source (e.g., coal, natural gas, renewable sources), and corresponding emissions of gases like CO2, CH4, and N2O. We apply machine learning techniques to uncover patterns and develop models capable of forecasting emissions based on energy consumption trends. This approach not only provides insights into current emission levels but also allows for scenario analysis to inform energy policy and sustainability efforts.

### Import and Process Data

In [23]:
def annual_growth(row, years):
    min_year = years["min"]
    max_year = years["max"]
    row["Indicator Name"] = row["Indicator Name"] + " - [annual growth %]"
    for year in range(max_year, min_year, -1):
        if not np.isnan(row[str(year)]) and not np.isnan(row[str(year - 1)]):
            row[str(year)] = 100 * (float(row[str(year)]) -
                                    float(row[str(year - 1)])) / abs(float(row[str(year - 1)]))
        else:
            row[str(year)] = np.nan
    row[str(min_year)] = np.nan
    return row


In [24]:

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'


def relabel(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan


def relabel_array(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return [1, 0, 0]
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return [0, 1, 0]
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return [0, 0, 1]
    else:
        return [np.nan]


def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index is not 0:
            boundaries.append(
                (prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries


<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/2s/yq898nms72b5qxqbwz_hx8j80000gn/T/ipykernel_29042/3838281611.py:31: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if index is not 0:


In [25]:


years = {"min": 1960, "max": 2019}

df_raw = read_csv("data/italy-raw-data.csv")
df_raw_growth = DataFrame(data=[row if "growth" in row["Indicator Name"] else annual_growth(
    row, years) for index, row in df_raw.iterrows()])

nodes = ['Pop', 'Urb', 'GDP', 'EC', 'FFEC', 'REC', 'EI', 'CO2', 'CH4', 'N2O']
df_growth = df_raw_growth.transpose().iloc[4:]
df_growth.columns = nodes
TIERS_NUM = 3

In [26]:

new_columns = {}
boundaries_map = {}
for i, content in enumerate(df_growth.items()):
    (label, series) = content
    values = np.sort(
        np.array([x for x in series.tolist() if not np.isnan(x)], dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    new_columns[label] = [relabel(value, boundaries)
                          for value in series.tolist()]
    boundaries_map[label] = boundaries

df = DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)

In [27]:

def get_joint_distribution(states):
    joint_dist = np.array(states[0])
    for state in states[1:]:
        joint_dist = np.outer(joint_dist, np.array(state))
    joint_dist = joint_dist.flatten()

    return joint_dist


def process_per_node_parsed(df, inputs, outputs, parsed=False):
    data_input = []
    data_output = []
    index = 0
    for row in df.iterrows():
        dt = row[1]
        inpt = []
        if parsed:
            parsed_inpts = []
            for i in inputs:
                value = relabel_array(dt[i], boundaries_map[i])
                parsed_inpts.append(value)
            inpt = get_joint_distribution(parsed_inpts)
        else:
            for i in inputs:
                # get from df_growth the row index and column i
                inpt.append(df_growth.loc[str(index + 1960), i])

        outpt = []
        parsed_outpts = []
        for i in outputs:
            value = relabel_array(dt[i], boundaries_map[i])
            parsed_outpts.append(value)
        outpt = get_joint_distribution(parsed_outpts)

        # if full input and full output
        if not any(np.isnan(inpt)) and not any(np.isnan(outpt)):
            data_input.append(inpt)
            data_output.append(outpt)
        index += 1
    return data_input, data_output

### CO2

#### Define Model CO2

In [68]:
def start_model_co2():
  encoded_size = 3
  dim_y = 3
  encoder = keras.layers.Identity()
  n_comp = 55

  nodes = [
    {'name': 'INPUT', 'type': 'input', 'dim_x': 3, 'dim_y': 3},
    {'name': 'EC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'FFEC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'REC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'CO2', 'dim_x': 9, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55}
  ]

  edges = [
    ('INPUT', 'EC'),
    ('EC', 'FFEC'),
    ('EC', 'REC'),
    ('REC', 'CO2'),
    ('FFEC', 'CO2'),
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
  )
  return model

#### Learning prototypes and Sigma

In [69]:
model = start_model_co2()

In [70]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [71]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CO2'], boundaries_map['CO2'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [72]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [73]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=900, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/900
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("Reshape:0", shape=(1, 9), dtype=float32)
Node: CO2, rho_x shape: (1, 1, 10), rho_x_values: Tensor("concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_2_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: CO2, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_2_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_2_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: CO2, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_2_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1855 - sparse_categorical_accuracy: 0.4184    
Epoch 2/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - loss: 1.1357 - sparse_categorical_accuracy: 0.3301  
Epoch 3/900
48/48 ━━━━━━━━━━━

#### Results

##### Train Result

In [74]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_2_1/Reshape:0", shape=(48, 9), dtype=float32)
Node: CO2, rho_x shape: (48, 1, 10), rho_x_values: Tensor("kdm_graph_model_2_1/concat_3:0", shape=(48, 1, 10), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
              precision    recall  f1-score   support

           0       0.87      0.76      0.81        17
           1       0.65      0.87      0.74        15
           2       0.92      0.75      0.83        16

    accuracy                           0.79        48
   macro avg       0.81      0.79      0.79        48
weighted avg       0.82      0.79      0.80        48



##### Test Result

In [75]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_2_1/Reshape:0", shape=(None, None), dtype=float32)
Node: CO2, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_2_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.50      0.33      0.40         3
           2       0.50      0.50      0.50         2

    accuracy                           0.50         6
   macro avg       0.50      0.61      0.52         6
weighted avg       0.50      0.50      0.48         6



##### Full Result

In [76]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.82      0.78      0.80        18
           1       0.64      0.78      0.70        18
           2       0.87      0.72      0.79        18

    accuracy                           0.76        54
   macro avg       0.78      0.76      0.76        54
weighted avg       0.78      0.76      0.76        54



#### Learning after initializing some prototypes (EC)

In [94]:
model = start_model_co2()

In [95]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

Initialize EC node

In [96]:
df_growth.head()

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,1.993928,2.836401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,0.668383,1.498807,7.486419,12.0622,2.344018,NaN,5.072614,13.892428,NaN,NaN
1962,0.676623,1.506833,5.487478,13.064053,1.933224,NaN,5.753948,17.588694,NaN,NaN
1963,0.729553,1.551287,4.842052,11.188621,-0.167728,NaN,2.519427,12.51158,NaN,NaN
1964,0.822624,1.636027,1.955533,9.110076,1.075163,NaN,0.631028,6.78298,NaN,NaN


In [97]:
input_EC = []
output_EC = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['EC'], boundaries_map['EC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_EC.append(inpt)
        value = get_joint_distribution([value_1])
        output_EC.append(value)
    index += 1


In [98]:
X_train, X_test, y_train, y_test = train_test_split(input_EC, output_EC, test_size=0.2, random_state=42)

In [99]:
input_EC_processed = X_train
output_EC_processed = y_train

In [100]:
for i in range(55 - len(input_EC_processed)):
    input_EC_processed.append(np.random.normal(0, 1, 3))
    output_EC_processed.append(np.random.normal(0, 1, 3))


In [101]:
print(len(input_EC_processed))

55


In [102]:
# Complete array to have 55 with random normal elements


model.init_components(
    np.array(input_EC_processed),
    np.array(output_EC_processed),
    node_name='EC'
)

In [103]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CO2'], boundaries_map['CO2'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [104]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [106]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=900, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - loss: 0.6785 - sparse_categorical_accuracy: 0.7142
Epoch 2/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step - loss: 0.6727 - sparse_categorical_accuracy: 0.5968
Epoch 3/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 0.6420 - sparse_categorical_accuracy: 0.6545   
Epoch 4/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step - loss: 0.8001 - sparse_categorical_accuracy: 0.5024  
Epoch 5/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - loss: 0.7664 - sparse_categorical_accuracy: 0.5417  
Epoch 6/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step - loss: 0.8004 - sparse_categorical_accuracy: 0.6053  
Epoch 7/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 0.6369 - sparse_categorical_accuracy: 0.7112
Epoch 8/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - loss: 0.6457 - sparse_categorical_accuracy: 0.7091
Epoch 9/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.6268 - sparse_categorical_accuracy: 0.6664
Epoch 10/900
48/48 ━━━━━━━━━━━━━━━━━━

#### Results

##### Train Result

In [107]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_4_1/Reshape:0", shape=(48, 9), dtype=float32)
Node: CO2, rho_x shape: (48, 1, 10), rho_x_values: Tensor("kdm_graph_model_4_1/concat_3:0", shape=(48, 1, 10), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
              precision    recall  f1-score   support

           0       1.00      0.71      0.83        17
           1       0.75      1.00      0.86        15
           2       0.88      0.88      0.88        16

    accuracy                           0.85        48
   macro avg       0.88      0.86      0.85        48
weighted avg       0.88      0.85      0.85        48



##### Test Result

In [108]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CO2
merged probs
Tensor("kdm_graph_model_4_1/Reshape:0", shape=(None, None), dtype=float32)
Node: CO2, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_4_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.50      0.33      0.40         3
           2       0.50      0.50      0.50         2

    accuracy                           0.50         6
   macro avg       0.50      0.61      0.52         6
weighted avg       0.50      0.50      0.48         6



##### Full Result

In [109]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
              precision    recall  f1-score   support

           0       0.93      0.72      0.81        18
           1       0.73      0.89      0.80        18
           2       0.83      0.83      0.83        18

    accuracy                           0.81        54
   macro avg       0.83      0.81      0.82        54
weighted avg       0.83      0.81      0.82        54



### N2O

#### Define Model N2O

In [110]:
def start_model_n2o():
  encoded_size = 3
  dim_y = 3
  encoder = keras.layers.Identity()
  n_comp = 55

  nodes = [
    {'name': 'INPUT', 'type': 'input', 'dim_x': 3, 'dim_y': 3},
    {'name': 'EC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'FFEC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'REC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'N2O', 'dim_x': 9, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55}
  ]

  edges = [
    ('INPUT', 'EC'),
    ('EC', 'FFEC'),
    ('EC', 'REC'),
    ('REC', 'N2O'),
    ('FFEC', 'N2O'),
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
  )
  return model

#### Learning Prototypes and Sigma

In [82]:
model = start_model_n2o()

In [83]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [84]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['N2O'], boundaries_map['N2O'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [85]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [87]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=900, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - loss: 0.8334 - sparse_categorical_accuracy: 0.5036
Epoch 2/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 0.8758 - sparse_categorical_accuracy: 0.5339
Epoch 3/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - loss: 0.8210 - sparse_categorical_accuracy: 0.4970   
Epoch 4/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9516 - sparse_categorical_accuracy: 0.3176     
Epoch 5/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - loss: 0.9091 - sparse_categorical_accuracy: 0.4230   
Epoch 6/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.9268 - sparse_categorical_accuracy: 0.4455   
Epoch 7/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8842 - sparse_categorical_accuracy: 0.4978    
Epoch 8/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - loss: 0.8607 - sparse_categorical_accuracy: 0.5909
Epoch 9/900
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step - loss: 0.8986 - sparse_categorical_accuracy: 0.5284   
Epoch 10/900
34/34 ━━━━━━━━━━

#### Results

##### Train Result

In [ ]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_11_1/Reshape:0", shape=(34, 9), dtype=float32)
Node: N2O, rho_x shape: (34, 1, 10), rho_x_values: Tensor("kdm_graph_model_11_1/concat_3:0", shape=(34, 1, 10), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
              precision    recall  f1-score   support

           0       0.71      0.91      0.80        11
           1       0.67      0.33      0.44        12
           2       0.64      0.82      0.72        11

    accuracy                           0.68        34
   macro avg       0.67      0.69      0.65        34
weighted avg       0.67      0.68      0.65        34



##### Test Result

In [ ]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_11_1/Reshape:0", shape=(None, None), dtype=float32)
Node: N2O, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_11_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1

    accuracy                           0.25         4
   macro avg       0.33      0.17      0.22         4
weighted avg       0.50      0.25      0.33         4



##### Full Result

In [ ]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
              precision    recall  f1-score   support

           0       0.73      0.85      0.79        13
           1       0.50      0.31      0.38        13
           2       0.60      0.75      0.67        12

    accuracy                           0.63        38
   macro avg       0.61      0.63      0.61        38
weighted avg       0.61      0.63      0.61        38



#### Learning after initializing some prototypes (EC and FFEC)

In [111]:
model = start_model_n2o()

In [112]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

Initialize EC and FFEC node

In [113]:
df_growth.head()

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,1.993928,2.836401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,0.668383,1.498807,7.486419,12.0622,2.344018,NaN,5.072614,13.892428,NaN,NaN
1962,0.676623,1.506833,5.487478,13.064053,1.933224,NaN,5.753948,17.588694,NaN,NaN
1963,0.729553,1.551287,4.842052,11.188621,-0.167728,NaN,2.519427,12.51158,NaN,NaN
1964,0.822624,1.636027,1.955533,9.110076,1.075163,NaN,0.631028,6.78298,NaN,NaN


In [117]:
input_EC = []
output_EC = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['EC'], boundaries_map['EC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_EC.append(inpt)
        value = get_joint_distribution([value_1])
        output_EC.append(value)
    index += 1


In [118]:
X_train_EC, X_test_EC, y_train_EC, y_test_EC = train_test_split(input_EC, output_EC, test_size=0.2, random_state=42)

In [119]:
input_FFEC = []
output_FFEC = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = relabel_array(dt['EC'], boundaries_map['EC'])
    value_1 = relabel_array(dt['FFEC'], boundaries_map['FFEC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_FFEC.append(inpt)
        value = get_joint_distribution([value_1])
        output_FFEC.append(value)
    index += 1


In [120]:
X_train_FFEC, X_test_FFEC, y_train_FFEC, y_test_FFEC = train_test_split(input_FFEC, output_FFEC, test_size=0.2, random_state=42)

In [121]:
input_EC_processed = X_train_EC
output_EC_processed = y_train_EC

In [122]:
for i in range(55 - len(input_EC_processed)):
    input_EC_processed.append(np.random.normal(0, 1, 3))
    output_EC_processed.append(np.random.normal(0, 1, 3))


In [123]:
input_FFEC_processed = X_train_FFEC
output_FFEC_processed = y_train_FFEC

In [124]:
for i in range(55 - len(input_FFEC_processed)):
    input_FFEC_processed.append(np.random.normal(0, 1, 3))
    output_FFEC_processed.append(np.random.normal(0, 1, 3))


In [125]:
model.init_components(
    np.array(input_EC_processed),
    np.array(output_EC_processed),
    node_name='EC'
)

In [127]:
model.init_components(
    np.array(input_FFEC_processed),
    np.array(output_FFEC_processed),
    node_name='FFEC'
)

In [128]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CO2'], boundaries_map['CO2'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [129]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [130]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=900, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/900
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("Reshape:0", shape=(1, 9), dtype=float32)
Node: N2O, rho_x shape: (1, 1, 10), rho_x_values: Tensor("concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_5_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: N2O, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_5_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_5_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: N2O, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_5_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.2206 - sparse_categorical_accuracy: 0.3098
Epoch 2/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1945 - sparse_categorical_accuracy: 0.4414
Epoch 3/900
48/48 ━━━━━━━━━━━━━━━━━━━

#### Results

##### Train Result

In [131]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_5_1/Reshape:0", shape=(48, 9), dtype=float32)
Node: N2O, rho_x shape: (48, 1, 10), rho_x_values: Tensor("kdm_graph_model_5_1/concat_3:0", shape=(48, 1, 10), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 280ms/step
              precision    recall  f1-score   support

           0       1.00      0.71      0.83        17
           1       0.68      1.00      0.81        15
           2       0.93      0.81      0.87        16

    accuracy                           0.83        48
   macro avg       0.87      0.84      0.84        48
weighted avg       0.88      0.83      0.84        48



##### Test Result

In [132]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
N2O
merged probs
Tensor("kdm_graph_model_5_1/Reshape:0", shape=(None, None), dtype=float32)
Node: N2O, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_5_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.50      0.33      0.40         3
           2       0.50      0.50      0.50         2

    accuracy                           0.50         6
   macro avg       0.50      0.61      0.52         6
weighted avg       0.50      0.50      0.48         6



##### Full Result

In [133]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
              precision    recall  f1-score   support

           0       0.93      0.72      0.81        18
           1       0.67      0.89      0.76        18
           2       0.88      0.78      0.82        18

    accuracy                           0.80        54
   macro avg       0.82      0.80      0.80        54
weighted avg       0.82      0.80      0.80        54



### CH4

#### Define Model CH4

In [134]:
def start_model_ch4():
  encoded_size = 3
  dim_y = 3
  encoder = keras.layers.Identity()
  n_comp = 55

  nodes = [
    {'name': 'INPUT', 'type': 'input', 'dim_x': 3, 'dim_y': 3},
    {'name': 'EC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'FFEC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'REC', 'dim_x': 3, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55},
    {'name': 'CH4', 'dim_x': 9, 'dim_y': 3, 'kernel': kdm.layers.CosineKernelLayer(), 'n_comp': 55}
  ]

  edges = [
    ('INPUT', 'EC'),
    ('EC', 'FFEC'),
    ('EC', 'REC'),
    ('REC', 'CH4'),
    ('FFEC', 'CH4'),
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
  )
  return model

#### Learning Prototypes and Sigma

In [103]:
model = start_model_ch4()

In [104]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [105]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CH4'], boundaries_map['CH4'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [106]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [107]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=1200, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/1200
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("Reshape:0", shape=(1, 9), dtype=float32)
Node: CH4, rho_x shape: (1, 1, 10), rho_x_values: Tensor("concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("kdm_graph_model_14_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: CH4, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_14_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("kdm_graph_model_14_1/Reshape:0", shape=(1, 9), dtype=float32)
Node: CH4, rho_x shape: (1, 1, 10), rho_x_values: Tensor("kdm_graph_model_14_1/concat_3:0", shape=(1, 1, 10), dtype=float32)
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1977 - sparse_categorical_accuracy: 0.2370
Epoch 2/1200
34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1631 - sparse_categorical_accuracy: 0.1895     
Epoch 3/1200
34/34 ━━━━━━━

#### Results

##### Train Result

In [108]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("kdm_graph_model_14_1/Reshape:0", shape=(34, 9), dtype=float32)
Node: CH4, rho_x shape: (34, 1, 10), rho_x_values: Tensor("kdm_graph_model_14_1/concat_3:0", shape=(34, 1, 10), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       0.92      1.00      0.96        12
           2       1.00      0.90      0.95        10

    accuracy                           0.97        34
   macro avg       0.97      0.97      0.97        34
weighted avg       0.97      0.97      0.97        34



##### Test Result

In [109]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("kdm_graph_model_14_1/Reshape:0", shape=(None, None), dtype=float32)
Node: CH4, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_14_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1
           2       1.00      0.50      0.67         2

    accuracy                           0.50         4
   macro avg       0.50      0.50      0.44         4
weighted avg       0.62      0.50      0.50         4



##### Full Result

In [110]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       0.86      0.92      0.89        13
           2       1.00      0.83      0.91        12

    accuracy                           0.92        38
   macro avg       0.93      0.92      0.92        38
weighted avg       0.93      0.92      0.92        38



#### Learning after initializing some prototypes (EC and FFEC)

In [137]:
model = start_model_ch4()

In [138]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-2),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

Initialize EC, FFEC and REC node

In [139]:
df_growth.head()

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,1.993928,2.836401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,0.668383,1.498807,7.486419,12.0622,2.344018,NaN,5.072614,13.892428,NaN,NaN
1962,0.676623,1.506833,5.487478,13.064053,1.933224,NaN,5.753948,17.588694,NaN,NaN
1963,0.729553,1.551287,4.842052,11.188621,-0.167728,NaN,2.519427,12.51158,NaN,NaN
1964,0.822624,1.636027,1.955533,9.110076,1.075163,NaN,0.631028,6.78298,NaN,NaN


In [170]:
input_EC = []
output_EC = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['EC'], boundaries_map['EC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_EC.append(inpt)
        value = get_joint_distribution([value_1])
        output_EC.append(value)
    index += 1


In [171]:
X_train_EC, X_test_EC, y_train_EC, y_test_EC = train_test_split(input_EC, output_EC, test_size=0.2, random_state=42)

In [172]:
input_FFEC = []
output_FFEC = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = relabel_array(dt['EC'], boundaries_map['EC'])
    value_1 = relabel_array(dt['FFEC'], boundaries_map['FFEC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_FFEC.append(inpt)
        value = get_joint_distribution([value_1])
        output_FFEC.append(value)
    index += 1


In [173]:
X_train_FFEC, X_test_FFEC, y_train_FFEC, y_test_FFEC = train_test_split(input_FFEC, output_FFEC, test_size=0.2, random_state=42)

In [174]:
df_growth.tail()

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
2015,-0.096376,0.325701,0.875477,2.786129,1.733165,-3.355742,1.888872,NaN,NaN,NaN
2016,-0.169884,0.246127,1.451875,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017,-0.149861,0.262999,1.868715,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018,-0.190064,0.228198,0.966058,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
print(boundaries_map['REC'])

[(-14.972188607586867, 2.0282717980738227), (2.0282717980738227, 11.505355728371361), (11.505355728371361, 23.954240592358776)]


In [176]:
input_REC = []
output_REC = []
for row in df_growth.iterrows():
    dt = row[1]

    inpt = relabel_array(dt['EC'], boundaries_map['EC'])
    value_1 = relabel_array(dt['REC'], boundaries_map['REC'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        input_REC.append(inpt)
        value = get_joint_distribution([value_1])
        output_REC.append(value)


In [177]:
X_train_REC, X_test_REC, y_train_REC, y_test_REC = train_test_split(input_REC, output_REC, test_size=0.2, random_state=42)

In [178]:
input_EC_processed = X_train_EC
output_EC_processed = y_train_EC

In [179]:
for i in range(55 - len(input_EC_processed)):
    input_EC_processed.append(np.random.normal(0, 1, 3))
    output_EC_processed.append(np.random.normal(0, 1, 3))


In [180]:
input_FFEC_processed = X_train_FFEC
output_FFEC_processed = y_train_FFEC

In [181]:
for i in range(55 - len(input_FFEC_processed)):
    input_FFEC_processed.append(np.random.normal(0, 1, 3))
    output_FFEC_processed.append(np.random.normal(0, 1, 3))


In [182]:
input_REC_processed = X_train_REC
output_REC_processed = y_train_REC

In [183]:
for i in range(55 - len(input_REC_processed)):
    input_REC_processed.append(np.random.normal(0, 1, 3))
    output_REC_processed.append(np.random.normal(0, 1, 3))


In [184]:
model.init_components(
    np.array(input_EC_processed),
    np.array(output_EC_processed),
    node_name='EC'
)

In [185]:
model.init_components(
    np.array(input_FFEC_processed),
    np.array(output_FFEC_processed),
    node_name='FFEC'
)

In [186]:
model.init_components(
    np.array(input_REC_processed),
    np.array(output_REC_processed),
    node_name='REC'
)

In [187]:

X = []
y = []
y_int = []
index = 0
for row in df_growth.iterrows():
    dt = row[1]
    inpt = [df_growth.loc[str(index + 1960), 'Pop'], df_growth.loc[str(
        index + 1960), 'Urb'], df_growth.loc[str(index + 1960), 'GDP']]
    value_1 = relabel_array(dt['CO2'], boundaries_map['CO2'])
    if not any(np.isnan(inpt)) and not any(np.isnan(value_1)):
        X.append(inpt)
        value = get_joint_distribution([value_1])
        
        y.append(value)
        y_int.append(list(value).index(1))
    index += 1


In [188]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_int, test_size=0.1, random_state=42)

In [191]:
history = model.fit(
    {'INPUT': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=900, 
    verbose=1,  # Detailed logging
    shuffle=True  # Shuffle the data
)

Epoch 1/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - loss: 0.4603 - sparse_categorical_accuracy: 0.7926
Epoch 2/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - loss: 0.6041 - sparse_categorical_accuracy: 0.6790
Epoch 3/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - loss: 0.6878 - sparse_categorical_accuracy: 0.6387  
Epoch 4/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - loss: 0.4570 - sparse_categorical_accuracy: 0.7834
Epoch 5/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 703us/step - loss: 0.4423 - sparse_categorical_accuracy: 0.8375
Epoch 6/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 0.4995 - sparse_categorical_accuracy: 0.7865
Epoch 7/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 0.5385 - sparse_categorical_accuracy: 0.7920
Epoch 8/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - loss: 0.5110 - sparse_categorical_accuracy: 0.8079
Epoch 9/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 0.5005 - sparse_categorical_accuracy: 0.7283
Epoch 10/900
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7

#### Results

##### Train Result

In [192]:
y_pred = model.predict({'INPUT': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
              precision    recall  f1-score   support

           0       0.86      0.71      0.77        17
           1       0.62      1.00      0.77        15
           2       1.00      0.62      0.77        16

    accuracy                           0.77        48
   macro avg       0.83      0.78      0.77        48
weighted avg       0.83      0.77      0.77        48



##### Test Result

In [193]:
y_pred = model.predict({'INPUT': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

node
INPUT
entro
INPUT
node
EC
node
FFEC
node
REC
node
CH4
merged probs
Tensor("kdm_graph_model_8_1/Reshape:0", shape=(None, None), dtype=float32)
Node: CH4, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_8_1/concat_3:0", shape=(None, 1, None), dtype=float32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.67      0.67      0.67         3
           2       1.00      0.50      0.67         2

    accuracy                           0.67         6
   macro avg       0.72      0.72      0.67         6
weighted avg       0.75      0.67      0.67         6



##### Full Result

In [194]:
y_pred = model.predict({'INPUT': np.array(X)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_int, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

           0       0.81      0.72      0.76        18
           1       0.63      0.94      0.76        18
           2       1.00      0.61      0.76        18

    accuracy                           0.76        54
   macro avg       0.81      0.76      0.76        54
weighted avg       0.81      0.76      0.76        54



## Iris Dataset Classification

In this section, we apply Kernel Density Matrices (KDM) to the classic IRIS dataset to demonstrate their effectiveness in classification tasks. The IRIS dataset, which consists of measurements of sepal length, sepal width, petal length, and petal width for three species of iris flowers, serves as a benchmark for evaluating the performance of various classification algorithms.

Using KDM, we estimate the probability density functions for each class (species) based on the feature measurements. This non-parametric approach allows us to capture the underlying distribution of the data more accurately, leading to improved classification performance. By comparing the results with other common classification methods, we highlight the strengths and versatility of KDM in handling real-world datasets with continuous features.

### Import and Process Data

In [195]:
df = read_csv('data/iris.csv')

In [196]:
df['type'] = df['type'].apply(lambda x: [1,0,0] if x == 'Iris-setosa' else [0,1,0] if x == 'Iris-versicolor' else [0,0,1])

### Define Iris Model

In [197]:
def start_iris_model():
  encoded_size = 4
  dim_y = 3
  encoder = keras.layers.Identity()
  n_comp = 150

  nodes = [
    {'name': 'input', 'type': 'input', 'dim_x': 4, 'dim_y': 4, 'n_comp': 150},
    {'name': 'output', 'dim_x': 4, 'dim_y': 3, 'kernel': CosineKernelLayer(), 'n_comp': 150},
  ]

  edges = [
    ('input', 'output'),
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
  )
  return model

### Train The Model

In [198]:
model = start_iris_model()

In [199]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-4),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [200]:
data = []
for row in df.iterrows():
    y = row[1]['type']
    x = [row[1]['sepal length'], row[1]['sepal width'], row[1]['petal length'], row[1]['petal width']]
    data.append((x, y))
    

In [201]:
input = np.array([x for x, y in data])
output = np.array([y for x, y in data])
y_out = np.array([int(np.argmax(y)) for y in output])

In [202]:
X_train, X_test, y_train, y_test = train_test_split(
    input, y_out, test_size=0.3, random_state=1998)

In [203]:
history = model.fit(
    {'input': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=300, 
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/300
node
input
entro
input
node
output
node
input
entro
input
node
output
node
input
entro
input
node
output
 1/94 ━━━━━━━━━━━━━━━━━━━━ 34s 369ms/step - loss: 1.0986 - sparse_categorical_accuracy: 0.0000e+00node
input
entro
input
node
output
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0872 - sparse_categorical_accuracy: 0.5126 - val_loss: 1.0212 - val_sparse_categorical_accuracy: 0.6364
Epoch 2/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 0.9922 - sparse_categorical_accuracy: 0.6154 - val_loss: 0.9346 - val_sparse_categorical_accuracy: 0.6364
Epoch 3/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - loss: 0.8926 - sparse_categorical_accuracy: 0.6665 - val_loss: 0.8395 - val_sparse_categorical_accuracy: 0.6364
Epoch 4/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step - loss: 0.7882 - sparse_categorical_accuracy: 0.6915 - val_loss: 0.7645 - val_sparse_categorical_accuracy: 0.6364
Epoch 5/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - loss: 0.7376 - sparse_categorical_accurac

### Results

#### Train Results

In [204]:
y_pred = model.predict({'input': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
input
entro
input
node
output
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/stepnode
input
entro
input
node
output
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.83      0.69      0.75        35
           2       0.76      0.87      0.81        39

    accuracy                           0.85       105
   macro avg       0.86      0.85      0.85       105
weighted avg       0.85      0.85      0.85       105



#### Test Results

In [205]:
y_pred = model.predict({'input': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        19
           1       0.88      0.47      0.61        15
           2       0.59      0.91      0.71        11

    accuracy                           0.80        45
   macro avg       0.80      0.79      0.77        45
weighted avg       0.84      0.80      0.79        45



#### Full Results

In [206]:
y_pred = model.predict({'input': np.array(input)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_out, y_pred_bool))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        50
           1       0.84      0.62      0.71        50
           2       0.71      0.88      0.79        50

    accuracy                           0.83       150
   macro avg       0.84      0.83      0.83       150
weighted avg       0.84      0.83      0.83       150



### Train The Model after initializing 

In [266]:
model = start_iris_model()

In [267]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-4),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [268]:
data = []
for row in df.iterrows():
    y = row[1]['type']
    x = [row[1]['sepal length'], row[1]['sepal width'], row[1]['petal length'], row[1]['petal width']]
    data.append((x, y))
    

In [269]:
inpt = np.array([x for x, y in data])
outpt = np.array([y for x, y in data])

# only take 70% of the data
inpt = inpt[:int(0.7 * len(inpt))]
outpt = outpt[:int(0.7 * len(outpt))]


In [270]:
processed_inpt = list(inpt)
processed_outpt = list(outpt)

In [271]:
for i in range(150 - len(processed_inpt)):
    processed_inpt.append(np.random.normal(0, 1, 4))
    processed_outpt.append(np.random.normal(0, 1, 3))

In [272]:
model.init_components(
    np.array(processed_inpt),
    np.array(processed_outpt),
    node_name='output'
)

In [273]:
input = np.array([x for x, y in data])
output = np.array([y for x, y in data])
y_out = np.array([int(np.argmax(y)) for y in output])

In [274]:
X_train, X_test, y_train, y_test = train_test_split(
    input, y_out, test_size=0.3, random_state=1998)

In [275]:
history = model.fit(
    {'input': np.array(X_train)},  # Your training data
    np.array(y_train),  # Your training labels
    batch_size=1,
    epochs=300, 
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/300
node
input
entro
input
node
output
node
input
entro
input
node
output
node
input
entro
input
node
output
 1/94 ━━━━━━━━━━━━━━━━━━━━ 58s 630ms/step - loss: 0.7053 - sparse_categorical_accuracy: 1.0000node
input
entro
input
node
output
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.1915 - sparse_categorical_accuracy: 0.3723 - val_loss: 1.0528 - val_sparse_categorical_accuracy: 0.3636
Epoch 2/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - loss: 1.0570 - sparse_categorical_accuracy: 0.4192 - val_loss: 1.0379 - val_sparse_categorical_accuracy: 0.3636
Epoch 3/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 561us/step - loss: 1.0459 - sparse_categorical_accuracy: 0.3360 - val_loss: 0.9982 - val_sparse_categorical_accuracy: 0.5455
Epoch 4/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step - loss: 0.9789 - sparse_categorical_accuracy: 0.6229 - val_loss: 0.9907 - val_sparse_categorical_accuracy: 0.6364
Epoch 5/300
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - loss: 0.9593 - sparse_categorical_accuracy: 0

### Results

#### Train Results

In [276]:
y_pred = model.predict({'input': np.array(X_train)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))

node
input
entro
input
node
output
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/stepnode
input
entro
input
node
output
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        31
           1       0.86      0.91      0.89        35
           2       0.92      0.87      0.89        39

    accuracy                           0.92       105
   macro avg       0.93      0.93      0.93       105
weighted avg       0.92      0.92      0.92       105



#### Test Results

In [277]:
y_pred = model.predict({'input': np.array(X_test)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.80      0.89        15
           2       0.79      1.00      0.88        11

    accuracy                           0.93        45
   macro avg       0.93      0.93      0.92        45
weighted avg       0.95      0.93      0.93        45



#### Full Results

In [278]:
y_pred = model.predict({'input': np.array(input)}, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_out, y_pred_bool))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.90      0.88      0.89        50
           2       0.88      0.90      0.89        50

    accuracy                           0.93       150
   macro avg       0.93      0.93      0.93       150
weighted avg       0.93      0.93      0.93       150



## Cancer Classification

In this section, we use Kernel Density Matrices (KDM) to predict lung diseases based on the Asia dataset by Lauritzen and Spiegelhalter (1988). This synthetic dataset explores the relationships between various factors such as tuberculosis, lung cancer, bronchitis, and visits to Asia.

##### Dataset Overview
The Asia dataset includes the following variables, each represented as a two-level factor with levels "yes" and "no":

- D (dyspnoea): Presence of shortness of breath.
- T (tuberculosis): Diagnosis of tuberculosis.
- L (lung cancer): Diagnosis of lung cancer.
- B (bronchitis): Diagnosis of bronchitis.
- A (visit to Asia): History of visiting Asia.
- S (smoking): History of smoking.
- X (chest X-ray): Result of a chest X-ray.
- E (tuberculosis versus lung cancer/bronchitis): Diagnostic differentiation between tuberculosis and lung cancer/bronchitis.

### Import and Process Data

In [288]:
df = read_csv('data/cancer.csv')

In [289]:
# parse all columns to 1 and 0
df = df.applymap(lambda x: [1,0]  if x == 'yes' else [0,1])

In [290]:
df['D'] = df['D'].apply(lambda x: 1 if x == [1,0] else 0)

In [291]:
df.head()

,A,S,T,L,B,E,X,D
0,"[0, 1]","[1, 0]","[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]",1
1,"[0, 1]","[1, 0]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]",0
2,"[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]","[1, 0]","[1, 0]",1
3,"[0, 1]","[0, 1]","[0, 1]","[0, 1]","[1, 0]","[0, 1]","[0, 1]",1
4,"[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]","[0, 1]",1


In [292]:
X = df[['A', 'S']]
y = df['D']

In [293]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [294]:
for col in X_train.columns:
    X_train[col] = X_train[col].apply(np.array)

In [295]:
for col in X_test.columns:
    X_test[col] = X_test[col].apply(np.array)

### Define Markov Model (DISCRETE)


In [296]:
def start_model():
  encoded_size = 2
  dim_y =2
  encoder = keras.layers.Identity()
  n_comp = 5000

  nodes = [
    {'name': 'input_A', 'type': 'input', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'input_S', 'type': 'input', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'T', 'dim_x': 2, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000},
    {'name': 'L', 'dim_x': 2, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000},
    {'name': 'B', 'dim_x': 2, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000},
    {'name': 'E', 'dim_x': 4, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000},
    {'name': 'output', 'dim_x': 4, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000}
  ]

  edges = [
    ('input_A', 'T'),
    ('T', 'E'),
    ('input_S', 'L'),
    ('input_S', 'B'),
    ('L', 'E'),
    ('E', 'output'),
    ('B', 'output')
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
  )
  return model

### Only Inference

In [297]:
model = start_model()

In [298]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-4),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [299]:
A = []
S = []
T = []
B = []
E = []
L = []
for row in df.iterrows():
  A.append(np.array(row[1]['A'], dtype=np.float32))
  S.append(np.array(row[1]['S'], dtype=np.float32))
  T.append(np.array(row[1]['T'], dtype=np.float32))
  B.append(np.array(row[1]['B'], dtype=np.float32))
  E.append(np.array(row[1]['E'], dtype=np.float32))
  L.append(np.array(row[1]['L'], dtype=np.float32))


In [300]:
model.init_components(np.array(A), np.array(T), node_name='T')
model.init_components(np.array(S), np.array(B), node_name='B')
model.init_components(np.array(S), np.array(L), node_name='L')
merged_T_L = []
for i in range(len(T)):
  data = []
  for j in range(len(T[i])):
    for k in range(len(L[j])):
      data.append(T[i][j]*L[i][k])
  merged_T_L.append(data)
model.init_components(np.array(merged_T_L), np.array(E), node_name='E')

In [301]:
X_dict = {}

In [302]:
for col in X.columns:
  data = []
  for row in X.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [303]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))


node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("Reshape:0", shape=(64, 4), dtype=float32)
Node: E, rho_x shape: (64, 1, 5), rho_x_values: Tensor("concat_3:0", shape=(64, 1, 5), dtype=float32)
node
output
merged probs
Tensor("Reshape_1:0", shape=(64, 4), dtype=float32)
Node: output, rho_x shape: (64, 1, 5), rho_x_values: Tensor("concat_4:0", shape=(64, 1, 5), dtype=float32)
node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_13_1/Reshape:0", shape=(64, 4), dtype=float32)
Node: E, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_13_1/concat_3:0", shape=(64, 1, 5), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_13_1/Reshape_1:0", shape=(64, 4), dtype=float32)
Node: output, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_13_1/concat_4:0", shape=(64, 1, 5), dtype=float32)
74/79 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/stepnode
input_A
en

/Users/cristhiancamilogomezneira/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/cristhiancamilogomezneira/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/cristhiancamilogomezneira/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

  ### Learning Prototypes 

#### Learn All the Prototypes

In [339]:
model = start_model()

In [340]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-3),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [341]:
X_dict = {}

In [342]:
for col in X_train.columns:
  data = []
  for row in X_train.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [343]:
history = model.fit(
    X_dict,
    np.array(y_train.values, dtype=np.int32),
    batch_size=3,
    epochs=50,  # Start with 100 epochs and use early stopping
    verbose=1,  # Detailed logging
    validation_split=0.3,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/50
node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("Reshape:0", shape=(None, None), dtype=float32)
Node: E, rho_x shape: (None, 1, None), rho_x_values: Tensor("concat_3:0", shape=(None, 1, None), dtype=float32)
node
output
merged probs
Tensor("Reshape_1:0", shape=(None, None), dtype=float32)
Node: output, rho_x shape: (None, 1, None), rho_x_values: Tensor("concat_4:0", shape=(None, 1, None), dtype=float32)
node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_17_1/Reshape:0", shape=(None, None), dtype=float32)
Node: E, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_17_1/concat_3:0", shape=(None, 1, None), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_17_1/Reshape_1:0", shape=(None, None), dtype=float32)
Node: output, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_17_1/concat_4:0", shape=(None, 1, None)

#### Results

##### Train Result

In [344]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))


node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_17_1/Reshape:0", shape=(64, 4), dtype=float32)
Node: E, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_17_1/concat_3:0", shape=(64, 1, 5), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_17_1/Reshape_1:0", shape=(64, 4), dtype=float32)
Node: output, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_17_1/concat_4:0", shape=(64, 1, 5), dtype=float32)
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/stepnode
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_17_1/Reshape:0", shape=(None, None), dtype=float32)
Node: E, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_17_1/concat_3:0", shape=(None, 1, None), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_17_1/Reshape_1:0", shape=(None, None), dtype=float32)
Node: output, rho_x shape: (None, 1, No

##### Test Result

In [345]:
X_dict = {}
for col in X_test.columns:
  data = []
  for row in X_test.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [346]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
              precision    recall  f1-score   support

           0       0.69      0.62      0.66       802
           1       0.61      0.68      0.64       698

    accuracy                           0.65      1500
   macro avg       0.65      0.65      0.65      1500
weighted avg       0.65      0.65      0.65      1500



##### Total Result

In [347]:
X_dict = {}
for col in X.columns:
  data = []
  for row in X.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [348]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

79/79 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
              precision    recall  f1-score   support

           0       0.69      0.64      0.66      2650
           1       0.62      0.67      0.65      2350

    accuracy                           0.66      5000
   macro avg       0.66      0.66      0.65      5000
weighted avg       0.66      0.66      0.66      5000



##### What are the prototypes?

In [349]:

T = model.layers_dict['T'].c_x

for i in range(10):
  idx = np.random.randint(0, T.shape[0])
  print(T[idx])

tf.Tensor([ 0.04889404 -0.02567089], shape=(2,), dtype=float32)
tf.Tensor([-0.00208111 -0.00626891], shape=(2,), dtype=float32)
tf.Tensor([ 0.05099672 -0.0074645 ], shape=(2,), dtype=float32)
tf.Tensor([0.07082434 0.02577856], shape=(2,), dtype=float32)
tf.Tensor([0.0127241  0.00220815], shape=(2,), dtype=float32)
tf.Tensor([ 0.05605415 -0.05546701], shape=(2,), dtype=float32)
tf.Tensor([-0.05560666  0.01716047], shape=(2,), dtype=float32)
tf.Tensor([0.01045879 0.03153862], shape=(2,), dtype=float32)
tf.Tensor([-0.00971348 -0.05705294], shape=(2,), dtype=float32)
tf.Tensor([-0.03823969 -0.11767478], shape=(2,), dtype=float32)


### Define Markov Model (CONTINUOUS)


In [351]:
def start_model():
  encoded_size = 2
  dim_y =2
  encoder = keras.layers.Identity()
  n_comp = 5000

  nodes = [
    {'name': 'input_A', 'type': 'input', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'input_S', 'type': 'input', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'T', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'L', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000},
    {'name': 'B', 'dim_x': 2, 'dim_y': 2, 'n_comp': 5000, 'kernel': CosineKernelLayer()},
    {'name': 'E', 'dim_x': 4, 'dim_y': 2, 'n_comp': 5000, 'kernel': CosineKernelLayer()},
    {'name': 'output', 'dim_x': 4, 'dim_y': 2, 'kernel': CosineKernelLayer(), 'n_comp': 5000}
  ]

  edges = [
    ('input_A', 'T'),
    ('T', 'E'),
    ('input_S', 'L'),
    ('input_S', 'B'),
    ('L', 'E'),
    ('E', 'output'),
    ('B', 'output')
  ]

  model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.3,
    nodes=nodes,
    edges=edges
  )
  return model

### Learning Prototypes and Sigma

In [352]:
model = start_model()

In [353]:
model.layers_dict['input_A'].trainable = False
model.layers_dict['input_S'].trainable = False

In [354]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
                  loss=losses.sparse_categorical_crossentropy,
                  metrics=[metrics.sparse_categorical_accuracy])

In [355]:
X_dict = {}
for col in X_train.columns:
  data = []
  for row in X_train.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [356]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [357]:
history = model.fit(
    X_dict,
    np.array(y_train.values, dtype=np.int32),
    batch_size=2,
    epochs=50,
    verbose=1,
    validation_split=0.2, 
    shuffle=True
)

Epoch 1/50
node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("Reshape:0", shape=(2, 4), dtype=float32)
Node: E, rho_x shape: (2, 1, 5), rho_x_values: Tensor("concat_3:0", shape=(2, 1, 5), dtype=float32)
node
output
merged probs
Tensor("Reshape_1:0", shape=(2, 4), dtype=float32)
Node: output, rho_x shape: (2, 1, 5), rho_x_values: Tensor("concat_4:0", shape=(2, 1, 5), dtype=float32)
node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_18_1/Reshape:0", shape=(2, 4), dtype=float32)
Node: E, rho_x shape: (2, 1, 5), rho_x_values: Tensor("kdm_graph_model_18_1/concat_3:0", shape=(2, 1, 5), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_18_1/Reshape_1:0", shape=(2, 4), dtype=float32)
Node: output, rho_x shape: (2, 1, 5), rho_x_values: Tensor("kdm_graph_model_18_1/concat_4:0", shape=(2, 1, 5), dtype=float32)
node
input_A
entro
input_A
node
input_S
entro
input_S
n

#### Results

##### Train Result

In [360]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred_bool))


node
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_18_1/Reshape:0", shape=(64, 4), dtype=float32)
Node: E, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_18_1/concat_3:0", shape=(64, 1, 5), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_18_1/Reshape_1:0", shape=(64, 4), dtype=float32)
Node: output, rho_x shape: (64, 1, 5), rho_x_values: Tensor("kdm_graph_model_18_1/concat_4:0", shape=(64, 1, 5), dtype=float32)
52/55 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/stepnode
input_A
entro
input_A
node
input_S
entro
input_S
node
T
node
L
node
B
node
E
merged probs
Tensor("kdm_graph_model_18_1/Reshape:0", shape=(None, None), dtype=float32)
Node: E, rho_x shape: (None, 1, None), rho_x_values: Tensor("kdm_graph_model_18_1/concat_3:0", shape=(None, 1, None), dtype=float32)
node
output
merged probs
Tensor("kdm_graph_model_18_1/Reshape_1:0", shape=(None, None), dtype=float32)
Node: output, rho_x shape: (None, 1, No

##### Test Result

In [361]:
X_dict = {}
for col in X_test.columns:
  data = []
  for row in X_test.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [362]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred_bool))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
              precision    recall  f1-score   support

           0       0.69      0.62      0.66       802
           1       0.61      0.68      0.64       698

    accuracy                           0.65      1500
   macro avg       0.65      0.65      0.65      1500
weighted avg       0.65      0.65      0.65      1500



##### Total Result

In [363]:
X_dict = {}
for col in X.columns:
  data = []
  for row in X.iterrows():
    data.append(row[1][col])
  X_dict['input_' +col] = np.array(data, dtype=np.float32)

In [364]:
y_pred = model.predict(X_dict, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

print(classification_report(y, y_pred_bool))

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
              precision    recall  f1-score   support

           0       0.69      0.64      0.66      2650
           1       0.62      0.67      0.65      2350

    accuracy                           0.66      5000
   macro avg       0.66      0.66      0.65      5000
weighted avg       0.66      0.66      0.66      5000



##### What are the prototypes?

In [365]:

T = model.layers_dict['T'].c_x

for i in range(10):
  idx = np.random.randint(0, T.shape[0])
  print(T[idx])

tf.Tensor([ 0.01411452 -0.04061897], shape=(2,), dtype=float32)
tf.Tensor([ 0.00906477 -0.02424773], shape=(2,), dtype=float32)
tf.Tensor([-0.06407552 -0.00701086], shape=(2,), dtype=float32)
tf.Tensor([0.0977449  0.03327943], shape=(2,), dtype=float32)
tf.Tensor([ 0.02235927 -0.06164381], shape=(2,), dtype=float32)
tf.Tensor([ 0.01842118 -0.02694007], shape=(2,), dtype=float32)
tf.Tensor([0.04039465 0.02246003], shape=(2,), dtype=float32)
tf.Tensor([0.06104185 0.04259691], shape=(2,), dtype=float32)
tf.Tensor([-0.04490811  0.04646909], shape=(2,), dtype=float32)
tf.Tensor([-0.04386998  0.04397426], shape=(2,), dtype=float32)
